In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Maritime Vessel Classification - Exploratory Analysis\n",
    "\n",
    "This notebook demonstrates how to use the Maritime Anomaly Detection pipeline for exploratory data analysis and model experimentation.\n",
    "\n",
    "## Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "from pathlib import Path\n",
    "\n",
    "# Add src to path\n",
    "sys.path.append(str(Path().absolute().parent / \"src\"))\n",
    "\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from loguru import logger\n",
    "\n",
    "# Import pipeline components\n",
    "from src.config import config\n",
    "from src.pipeline import MaritimePipeline\n",
    "from src.data import AISDataLoader, AISPreprocessor\n",
    "from src.features import VesselFeatureExtractor, TrajectoryKalmanFilter\n",
    "from src.utils import ModelEvaluator, ModelVisualizer\n",
    "\n",
    "# Configure plotting\n",
    "plt.style.use('default')\n",
    "sns.set_palette(\"husl\")\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Data Loading and Exploration\n",
    "\n",
    "Let's start by loading and exploring the AIS data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Update these paths for your data\n",
    "ZIP_FILE_PATH = \"../data/AIS_2024_10_24.zip\"\n",
    "CSV_FILE_NAME = \"AIS_2024_10_24.csv\"\n",
    "\n",
    "# Create data loader\n",
    "data_loader = AISDataLoader()\n",
    "\n",
    "# For this example, we'll work with sample data\n",
    "config.TEST_MODE = True\n",
    "\n",
    "# Extract and load data\n",
    "if not Path(\"../data/processed/sample_ais_data.csv\").exists():\n",
    "    csv_path = data_loader.extract_zip_file(ZIP_FILE_PATH, \"../data/raw/extracted\")\n",
    "    sample_path = data_loader.create_sample_data(csv_path, \"../data/processed/sample_ais_data.csv\")\n",
    "else:\n",
    "    sample_path = \"../data/processed/sample_ais_data.csv\"\n",
    "\n",
    "# Load the data\n",
    "df_raw = data_loader.load_csv_data(sample_path, test_mode=True)\n",
    "\n",
    "print(f\"Loaded {len(df_raw)} records with {len(df_raw.columns)} columns\")\n",
    "print(f\"Unique vessels: {df_raw['MMSI'].nunique()}\")\n",
    "print(f\"Date range: {df_raw['BaseDateTime'].min()} to {df_raw['BaseDateTime'].max()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Basic data exploration\n",
    "df_raw.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Data info\n",
    "df_raw.info()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Vessel type distribution\n",
    "plt.figure(figsize=(12, 6))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "vessel_counts = df_raw['VesselType'].value_counts().head(10)\n",
    "vessel_counts.plot(kind='bar')\n",
    "plt.title('Top 10 Vessel Types (Count)')\n",
    "plt.xlabel('Vessel Type')\n",
    "plt.ylabel('Count')\n",
    "plt.xticks(rotation=45)\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "vessel_props = df_raw['VesselType'].value_counts(normalize=True).head(10)\n",
    "vessel_props.plot(kind='pie', autopct='%1.1f%%')\n",
    "plt.title('Top 10 Vessel Types (Proportion)')\n",
    "plt.ylabel('')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Data Preprocessing\n",
    "\n",
    "Let's preprocess the data and see how it affects the dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Apply preprocessing\n",
    "preprocessor = AISPreprocessor()\n",
    "\n",
    "# First, basic validation\n",
    "df_validated = data_loader.validate_required_columns(df_raw)\n",
    "df_validated = data_loader.basic_data_validation(df_validated)\n",
    "\n",
    "print(f\"After validation: {len(df_validated)} records\")\n",
    "\n",
    "# Apply full preprocessing\n",
    "df_processed = preprocessor.preprocess_pipeline(df_validated)\n",
    "\n",
    "print(f\"After preprocessing: {len(df_processed)} records\")\n",
    "print(f\"Unique vessels: {df_processed['MMSI'].nunique()}\")\n",
    "print(f\"Unique trajectories: {df_processed['traj_id'].nunique()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare vessel type distributions before and after preprocessing\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))\n",
    "\n",
    "# Before\n",
    "df_raw['VesselType'].value_counts().plot(kind='bar', ax=ax1)\n",
    "ax1.set_title('Vessel Types - Before Preprocessing')\n",
    "ax1.set_xlabel('Vessel Type')\n",
    "ax1.set_ylabel('Count')\n",
    "ax1.tick_params(axis='x', rotation=45)\n",
    "\n",
    "# After\n",
    "df_processed['VesselType'].value_counts().plot(kind='bar', ax=ax2)\n",
    "ax2.set_title('Vessel Types - After Preprocessing')\n",
    "ax2.set_xlabel('Vessel Type')\n",
    "ax2.set_ylabel('Count')\n",
    "ax2.tick_params(axis='x', rotation=45)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Trajectory Analysis\n",
    "\n",
    "Let's examine vessel trajectories and apply Kalman filtering."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Trajectory statistics\n",
    "traj_stats = df_processed.groupby(['MMSI', 'traj_id']).size()\n",
    "\n",
    "print(f\"Trajectory length statistics:\")\n",
    "print(traj_stats.describe())\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "plt.hist(traj_stats, bins=30, alpha=0.7, edgecolor='black')\n",
    "plt.xlabel('Trajectory Length (number of points)')\n",
    "plt.ylabel('Frequency')\n",
    "plt.title('Distribution of Trajectory Lengths')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Apply Kalman filtering\n",
    "kalman_filter = TrajectoryKalmanFilter()\n",
    "df_smoothed = kalman_filter.process_all_trajectories(df_processed)\n",
    "\n",
    "# Validate smoothing quality\n",
    "quality_metrics = kalman_filter.validate_smoothing_quality(df_smoothed)\n",
    "print(\"Kalman filter quality metrics:\")\n",
    "for metric, value in quality_metrics.items():\n",
    "    print(f\"  {metric}: {value}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize smoothing effect\n",
    "# Pick a random trajectory to visualize\n",
    "sample_vessel = df_smoothed['MMSI'].iloc[0]\n",
    "sample_traj = df_smoothed['traj_id'].iloc[0]\n",
    "\n",
    "sample_data = df_smoothed[\n",
    "    (df_smoothed['MMSI'] == sample_vessel) & \n",
    "    (df_smoothed['traj_id'] == sample_traj)\n",
    "].copy().sort_values('BaseDateTime')\n",
    "\n",
    "if len(sample_data) > 5:\n",
    "    plt.figure(figsize=(12, 8))\n",
    "    \n",
    "    # Plot original vs smoothed coordinates\n",
    "    plt.subplot(2, 2, 1)\n",
    "    plt.plot(sample_data['LON'], sample_data['LAT'], 'r.-', alpha=0.7, label='Original')\n",
    "    plt.plot(sample_data['LON_smoothed'], sample_data['LAT_smoothed'], 'b.-', alpha=0.7, label='Smoothed')\n",
    "    plt.xlabel('Longitude')\n",
    "    plt.ylabel('Latitude')\n",
    "    plt.title('Trajectory: Original vs Smoothed')\n",
    "    plt.legend()\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    \n",
    "    # Plot speed over time\n",
    "    plt.subplot(2, 2, 2)\n",
    "    plt.plot(sample_data['BaseDateTime'], sample_data['SOG'], 'g.-')\n",
    "    plt.xlabel('Time')\n",
    "    plt.ylabel('Speed over Ground (knots)')\n",
    "    plt.title('Speed Profile')\n",
    "    plt.xticks(rotation=45)\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    \n",
    "    # Plot course over time\n",
    "    plt.subplot(2, 2, 3)\n",
    "    plt.plot(sample_data['BaseDateTime'], sample_data['COG'], 'm.-')\n",
    "    plt.xlabel('Time')\n",
    "    plt.ylabel('Course over Ground (degrees)')\n",
    "    plt.title('Course Profile')\n",
    "    plt.xticks(rotation=45)\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    \n",
    "    # Plot smoothing differences\n",
    "    plt.subplot(2, 2, 4)\n",
    "    lat_diff = np.abs(sample_data['LAT'] - sample_data['LAT_smoothed'])\n",
    "    lon_diff = np.abs(sample_data['LON'] - sample_data['LON_smoothed'])\n",
    "    plt.plot(sample_data['BaseDateTime'], lat_diff, 'r.-', label='LAT diff')\n",
    "    plt.plot(sample_data['BaseDateTime'], lon_diff, 'b.-', label='LON diff')\n",
    "    plt.xlabel('Time')\n",
    "    plt.ylabel('Absolute Difference')\n",
    "    plt.title('Smoothing Effect')\n",
    "    plt.xticks(rotation=45)\n",
    "    plt.legend()\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"Sample trajectory too short for visualization\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Feature Extraction\n",
    "\n",
    "Let's extract features from the processed trajectories."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Extract features\n",
    "feature_extractor = VesselFeatureExtractor()\n",
    "features_df, X, y = feature_extractor.create_feature_pipeline(df_smoothed)\n",
    "\n",
    "print(f\"Features extracted:\")\n",
    "print(f\"  Samples: {X.shape[0]}\")\n",
    "print(f\"  Features: {X.shape[1]}\")\n",
    "print(f\"  Classes: {len(np.unique(y))}\")\n",
    "\n",
    "print(f\"\\nFeature columns:\")\n",
    "for i, col in enumerate(X.columns):\n",
    "    print(f\"  {i+1:2d}. {col}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature distribution analysis\n",
    "# Select a few interesting features for visualization\n",
    "interesting_features = [\n",
    "    'Length', 'Width', 'Speed_mean', 'Speed_max', \n",
    "    'Total_distance_km', 'Duration_hr', 'Stop_ratio', 'High_speed_ratio'\n",
    "]\n",
    "\n",
    "available_features = [f for f in interesting_features if f in X.columns]\n",
    "\n",
    "if len(available_features) >= 4:\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "    axes = axes.ravel()\n",
    "    \n",
    "    for i, feature in enumerate(available_features[:4]):\n",
    "        X[feature].hist(bins=30, alpha=0.7, ax=axes[i])\n",
    "        axes[i].set_title(f'Distribution of {feature}')\n",
    "        axes[i].set_xlabel(feature)\n",
    "        axes[i].set_ylabel('Frequency')\n",
    "        axes[i].grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "else:\n",
    "    print(f\"Available features for plotting: {available_features}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature correlation analysis\n",
    "correlation_matrix = X.corr()\n",
    "\n",
    "plt.figure(figsize=(12, 10))\n",
    "mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))\n",
    "sns.heatmap(correlation_matrix, mask=mask, annot=False, cmap='coolwarm', center=0)\n",
    "plt.title('Feature Correlation Matrix')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Show highly correlated features\n",
    "high_corr = correlation_matrix.abs() > 0.8\n",
    "high_corr_pairs = []\n",
    "for i in range(len(correlation_matrix.columns)):\n",
    "    for j in range(i+1, len(correlation_matrix.columns)):\n",
    "        if high_corr.iloc[i, j]:\n",
    "            high_corr_pairs.append((\n",
    "                correlation_matrix.columns[i],\n",
    "                correlation_matrix.columns[j],\n",
    "                correlation_matrix.iloc[i, j]\n",
    "            ))\n",
    "\n",
    "if high_corr_pairs:\n",
    "    print(\"Highly correlated feature pairs (|correlation| > 0.8):\")\n",
    "    for feat1, feat2, corr in high_corr_pairs:\n",
    "        print(f\"  {feat1} <-> {feat2}: {corr:.3f}\")\n",
    "else:\n",
    "    print(\"No highly correlated feature pairs found\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Model Training and Evaluation\n",
    "\n",
    "Let's train the complete ensemble model and evaluate its performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create and run the complete pipeline\n",
    "pipeline = MaritimePipeline()\n",
    "\n",
    "# For demonstration, we'll use the already processed data\n",
    "# In practice, you would run: pipeline.run_complete_pipeline(ZIP_FILE_PATH, CSV_FILE_NAME)\n",
    "\n",
    "# Prepare sequence data\n",
    "sequences = pipeline.prepare_sequence_data(df_smoothed, features_df)\n",
    "print(f\"Sequence data shape: {sequences.shape}\")\n",
    "\n",
    "# Split and scale data\n",
    "(X_train, X_test, y_train, y_test, \n",
    " seq_train, seq_test, class_weights) = pipeline.split_and_scale_data(X.values, y.values, sequences)\n",
    "\n",
    "print(f\"Training set: {X_train.shape[0]} samples\")\n",
    "print(f\"Test set: {X_test.shape[0]} samples\")\n",
    "print(f\"Class weights: {class_weights}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Train the ensemble\n",
    "logger.info(\"Training ensemble model...\")\n",
    "ensemble = pipeline.train_ensemble(X_train, y_train, seq_train)\n",
    "\n",
    "print(\"Ensemble training completed!\")\n",
    "print(f\"Base models: {list(ensemble.base_models.keys())}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Make predictions\n",
    "predictions = ensemble.predict(X_test, seq_test)\n",
    "probabilities = ensemble.predict_proba(X_test, seq_test)\n",
    "\n",
    "# Convert labels back if needed\n",
    "if hasattr(pipeline.label_encoder, 'classes_'):\n",
    "    y_test_orig = pipeline.label_encoder.inverse_transform(y_test)\n",
    "    class_names = list(pipeline.label_encoder.classes_)\n",
    "else:\n",
    "    y_test_orig = y_test\n",
    "    class_names = None\n",
    "\n",
    "print(f\"Predictions shape: {predictions.shape}\")\n",
    "print(f\"Probabilities shape: {probabilities.shape}\")\n",
    "print(f\"Class names: {class_names}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate model performance\n",
    "evaluator = ModelEvaluator()\n",
    "results = evaluator.evaluate_model(\n",
    "    y_test_orig, predictions, probabilities, class_names, \"Ensemble\"\n",
    ")\n",
    "\n",
    "# Print detailed report\n",
    "evaluator.print_detailed_report(\"Ensemble\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Visualizations\n",
    "\n",
    "Let's create comprehensive visualizations of the model performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create visualizer\n",
    "visualizer = ModelVisualizer(save_plots=False)  # Don't save plots in notebook\n",
    "\n",
    "# Confusion matrix\n",
    "visualizer.plot_confusion_matrix(y_test_orig, predictions, class_names)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Classification report\n",
    "visualizer.plot_classification_report(y_test_orig, predictions, class_names)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ROC curves\n",
    "visualizer.plot_roc_curves(y_test_orig, probabilities, class_names)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature importance from ensemble\n",
    "importance_df = ensemble.get_base_model_importance()\n",
    "if not importance_df.empty:\n",
    "    visualizer.plot_feature_importance(importance_df, \"Base Model Importance\")\n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"No feature importance data available\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Complete evaluation dashboard\n",
    "visualizer.create_evaluation_dashboard(\n",
    "    y_test_orig, predictions, probabilities, class_names, \"Maritime Ensemble\"\n",
    ")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Analysis and Insights\n",
    "\n",
    "Let's analyze the results and extract insights."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Performance by vessel type\n",
    "from sklearn.metrics import classification_report\n",
    "report = classification_report(y_test_orig, predictions, output_dict=True)\n",
    "\n",
    "per_class_df = pd.DataFrame(report).T\n",
    "per_class_df = per_class_df[per_class_df.index.isin(class_names)]\n",
    "\n",
    "print(\"Performance by Vessel Type:\")\n",
    "print(per_class_df[['precision', 'recall', 'f1-score', 'support']].round(3))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Prediction confidence analysis\n",
    "max_probs = np.max(probabilities, axis=1)\n",
    "correct_predictions = (predictions == y_test_orig)\n",
    "\n",
    "plt.figure(figsize=(12, 5))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.hist([max_probs[correct_predictions], max_probs[~correct_predictions]], \n",
    "         bins=20, alpha=0.7, label=['Correct', 'Incorrect'], density=True)\n",
    "plt.xlabel('Maximum Prediction Probability')\n",
    "plt.ylabel('Density')\n",
    "plt.title('Prediction Confidence Distribution')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "confidence_bins = np.arange(0, 1.1, 0.1)\n",
    "binned_confidence = np.digitize(max_probs, confidence_bins)\n",
    "accuracy_by_confidence = []\n",
    "for i in range(1, len(confidence_bins)):\n",
    "    mask = binned_confidence == i\n",
    "    if mask.sum() > 0:\n",
    "        accuracy = correct_predictions[mask].mean()\n",
    "        accuracy_by_confidence.append(accuracy)\n",
    "    else:\n",
    "        accuracy_by_confidence.append(0)\n",
    "\n",
    "plt.plot(confidence_bins[1:], accuracy_by_confidence, 'bo-')\n",
    "plt.xlabel('Prediction Confidence (binned)')\n",
    "plt.ylabel('Accuracy')\n",
    "plt.title('Accuracy vs Prediction Confidence')\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"Average confidence for correct predictions: {max_probs[correct_predictions].mean():.3f}\")\n",
    "print(f\"Average confidence for incorrect predictions: {max_probs[~correct_predictions].mean():.3f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "\n",
    "This notebook demonstrated the complete maritime vessel classification pipeline, from data loading to model evaluation. Key takeaways:\n",
    "\n",
    "1. **Data Quality**: Preprocessing significantly reduces the dataset size but improves data quality\n",
    "2. **Feature Engineering**: The extracted features capture both static vessel characteristics and dynamic behavioral patterns\n",
    "3. **Model Performance**: The ensemble approach combines multiple model types for robust classification\n",
    "4. **Evaluation**: Comprehensive metrics and visualizations provide insights into model performance\n",
    "\n",
    "### Next Steps\n",
    "\n",
    "- Experiment with different feature engineering approaches\n",
    "- Try different ensemble configurations\n",
    "- Analyze prediction errors to identify improvement opportunities\n",
    "- Test on different geographic regions or time periods\n",
    "- Implement online learning for adapting to new data patterns"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save the trained model for later use\n",
    "model_path = pipeline.save_model(\"../data/models/notebook_trained_model.joblib\")\n",
    "print(f\"Model saved to: {model_path}\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}